# Models on the UCI datasets

In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot

In [2]:
from tensorflow.keras.utils import to_categorical

2022-03-05 17:50:34.430560: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


# Data loading utilities

In [3]:
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values
 
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        #print(data.shape)
        loaded.append(data)
        
    loaded = dstack(loaded)
    return loaded

# load train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y


# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [4]:
# Load the data
trainX, trainy, testX, testy = load_dataset()

(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [5]:
trainX, trainy, testX, testy = load_dataset()

(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


# Using Conv-BI-Directional LSTM

In [13]:
# LSTM Network Default input: [batch, timesteps, feature]
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess= tf.Session(config=config)


model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(trainX.shape[1], trainX.shape[2])))
model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, strides=2, activation='relu'))
model.add(tf.keras.layers.Reshape((-1,32*61)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, activation='relu')))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(trainy.shape[1], activation='softmax'))
#model.output_shape

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

2022-03-05 17:55:32.833454: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-05 17:55:32.834503: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.683GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-03-05 17:55:32.835439: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:02:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.683GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-03-05 17:55:32.835493: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-03-05 17:55:32.835527: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libc

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 124, 64)           2944      
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 61, 32)            6176      
_________________________________________________________________
reshape_2 (Reshape)          (None, 1, 1952)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32)                252032    
_________________________________________________________________
dense_4 (Dense)              (None, 64)                2112      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)               

In [14]:
filepath = 'BiLSTM_Best.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, 
                             monitor='val_acc',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')

callbacks = [checkpoint]

model.fit(trainX, trainy, batch_size=64, epochs=100,
            verbose=1, validation_data=(testX, testy),
         callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Epoch 1/100
7296/7352 [============================>.] - ETA: 0s - loss: 0.8278 - acc: 0.6502
Epoch 00001: val_acc improved from -inf to 0.79369, saving model to BiLSTM_Best.hdf5
7352/7352 [==============================] - 3s 370us/sample - loss: 0.8248 - acc: 0.6518 - val_loss: 0.5462 - val_acc: 0.7937
Epoch 2/100
7232/7352 [============================>.] - ETA: 0s - loss: 0.2530 - acc: 0.9045
Epoch 00002: val_acc improved from 0.79369 to 0.87241, saving model to BiLSTM_Best.hdf5
7352/7352 [==============================] - 1s 187us/sample - loss: 0.2510 - acc: 0.9052 - val_loss: 0.3468 - val_acc: 0.8724
Epoch 3/100
7040/7352 [===========================>..] - ETA: 0s - loss: 0.1524 - acc: 0.9429
Epoch 00003: val_acc improved from 0.87241 to 0.88972, saving model to BiLSTM_Best.hdf5
7352/7352 [==============================] - 1s 188us/sample - loss: 0.1518 - acc: 0.9433 - val_loss: 0.2640 - val_acc: 0.8897
Epoch 4/100
7104/7352 [=====

Epoch 30/100
7352/7352 [==============================] - ETA: 0s - loss: 0.0538 - acc: 0.9747
Epoch 00030: val_acc did not improve from 0.93485
7352/7352 [==============================] - 1s 178us/sample - loss: 0.0538 - acc: 0.9747 - val_loss: 0.5778 - val_acc: 0.9040
Epoch 31/100
7232/7352 [============================>.] - ETA: 0s - loss: 0.0628 - acc: 0.9688
Epoch 00031: val_acc did not improve from 0.93485
7352/7352 [==============================] - 1s 188us/sample - loss: 0.0626 - acc: 0.9687 - val_loss: 0.5978 - val_acc: 0.9070
Epoch 32/100
7352/7352 [==============================] - ETA: 0s - loss: 0.0566 - acc: 0.9720
Epoch 00032: val_acc did not improve from 0.93485
7352/7352 [==============================] - 1s 186us/sample - loss: 0.0566 - acc: 0.9720 - val_loss: 0.5313 - val_acc: 0.9043
Epoch 33/100
7040/7352 [===========================>..] - ETA: 0s - loss: 0.0490 - acc: 0.9754
Epoch 00033: val_acc did not improve from 0.93485
7352/7352 [============================

7232/7352 [============================>.] - ETA: 0s - loss: 0.0429 - acc: 0.9800
Epoch 00060: val_acc did not improve from 0.93485
7352/7352 [==============================] - 1s 190us/sample - loss: 0.0426 - acc: 0.9801 - val_loss: 0.6094 - val_acc: 0.9080
Epoch 61/100
7040/7352 [===========================>..] - ETA: 0s - loss: 0.0295 - acc: 0.9862
Epoch 00061: val_acc did not improve from 0.93485
7352/7352 [==============================] - 1s 182us/sample - loss: 0.0309 - acc: 0.9857 - val_loss: 0.7371 - val_acc: 0.9118
Epoch 62/100
7104/7352 [===========================>..] - ETA: 0s - loss: 0.0398 - acc: 0.9823
Epoch 00062: val_acc did not improve from 0.93485
7352/7352 [==============================] - 1s 182us/sample - loss: 0.0391 - acc: 0.9826 - val_loss: 0.6029 - val_acc: 0.9111
Epoch 63/100
7040/7352 [===========================>..] - ETA: 0s - loss: 0.0288 - acc: 0.9862
Epoch 00063: val_acc did not improve from 0.93485
7352/7352 [==============================] - 1s 182u

7232/7352 [============================>.] - ETA: 0s - loss: 0.0208 - acc: 0.9891
Epoch 00090: val_acc did not improve from 0.93485
7352/7352 [==============================] - 1s 187us/sample - loss: 0.0206 - acc: 0.9891 - val_loss: 0.7056 - val_acc: 0.9141
Epoch 91/100
7296/7352 [============================>.] - ETA: 0s - loss: 0.0243 - acc: 0.9881
Epoch 00091: val_acc did not improve from 0.93485
7352/7352 [==============================] - 1s 185us/sample - loss: 0.0242 - acc: 0.9882 - val_loss: 0.6383 - val_acc: 0.9152
Epoch 92/100
7352/7352 [==============================] - ETA: 0s - loss: 0.0150 - acc: 0.9946
Epoch 00092: val_acc did not improve from 0.93485
7352/7352 [==============================] - 1s 185us/sample - loss: 0.0150 - acc: 0.9946 - val_loss: 0.5976 - val_acc: 0.9165
Epoch 93/100
7168/7352 [============================>.] - ETA: 0s - loss: 0.0327 - acc: 0.9870
Epoch 00093: val_acc did not improve from 0.93485
7352/7352 [==============================] - 1s 184u

In [15]:
# Re-evaluate the model
model.load_weights(filepath)

loss, acc = model.evaluate(testX, testy, verbose=2)
print("LSTM model, accuracy: {:5.2f}%".format(100 * acc))

LSTM model, accuracy: 93.48%
